                        Project done by Michela Pirozzi MAT:732531 and Sara Ferioli MAT:733105

# LAB 3

### Libraries

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

### Functions

In [2]:
# Method that allow to load ateco csv and prepare the data for the join
def addingAteco():
    df_ateco = pd.read_csv("ateco.csv")
    df_ateco.Descrizione_ateco.replace(r'^.* - ' , '',regex=True, inplace = True,)
    df_ateco = df_ateco.drop("_id",axis=1)
    df_ateco.rename(columns = {"Descrizione_ateco":"SETTOREECONOMICODETTAGLIO"}, inplace = True)
    df_ateco['SETTOREECONOMICODETTAGLIO'] = df_ateco['SETTOREECONOMICODETTAGLIO'].str.upper()
    df_ateco.drop_duplicates(subset="SETTOREECONOMICODETTAGLIO",keep="first",inplace=True)
    return df_ateco

In [3]:
# Function to transform the columns ETA, nazionalita_transformed, genere_transformed in a unique column
# with the format: "ETA - nazionalita_transformed - genere_transformed"
# syntax:
# df: Dataframe
# column: column name of the new column
def trasformed_input(df,column):
    df[column] = df["Codice_ateco"].apply(str) + " - " + df["titolostudio_transformed"].apply(str)+ " - " + df["modalitalavoro_transformed"].apply(str)+ " - " + df["provincia_transformed"].apply(str)
    
    #Non eliminiamo Codice_ateco per poter predirre su di esso
    return df.drop(labels=['titolostudio_transformed','modalitalavoro_transformed','provincia_transformed'], axis=1)

In [4]:
# Function to retrive the original data from the trasformed ones
# syntax:
# df: Dataframe
# column: column name of the trasformed column
def inverse_trasformed_input(df,column):
    if type(df) is str:
        codice_ateco,titolostudio,modalitalavoro,provincia=df.split(" - ")
        print("Codice_ateco: "+codice_ateco+" titolostudio: "+titolostudio+" modalitalavoro: "+modalitalavoro+" provincia: "+provincia)
        return codice_ateco,titolostudio,modalitalavoro,provincia
    
    df[['Codice_ateco','titolostudio_transformed','modalitalavoro_transformed','provincia_transformed']] = df[column].str.split(" - ",expand=True,)
    return df.drop(labels =[column], axis=1)

In [5]:
# Function to retrive the original data from the trasformed ones
# syntax:
# value: trasfomed value
# return 'M' if value is 0 otherwise 'F'
def gender(value):
    if value=='0':
        return 'M'
    else:
        return 'F'

### Import data and analysis

In [6]:
# Load the dataframe
attivati = pd.read_csv("Rapporti_di_lavoro_attivati.csv")

In [7]:
# Load ateco csv and prepare the data for the join
ateco = addingAteco()
ateco

,Codice_ateco,SETTOREECONOMICODETTAGLIO
0,01,COLTIVAZIONI AGRICOLE E PRODUZIONE DI PRODOTTI...
1,01.1,COLTIVAZIONE DI COLTURE AGRICOLE NON PERMANENTI
2,01.11,"COLTIVAZIONE DI CEREALI (ESCLUSO IL RISO), LEG..."
3,01.11.1,COLTIVAZIONE DI CEREALI (ESCLUSO IL RISO)
5,01.11.2,COLTIVAZIONE DI SEMI OLEOSI
...,...,...
3100,96.09.09,ALTRE ATTIVITÀ DI SERVIZI PER LA PERSONA NCA
3101,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
3106,98,PRODUZIONE DI BENI E SERVIZI INDIFFERENZIATI P...
3107,98.1,PRODUZIONE DI BENI INDIFFERENZIATI PER USO PRO...


In [8]:
# Transformations of the strings in upper case, to make join
attivati['SETTOREECONOMICODETTAGLIO'] = attivati['SETTOREECONOMICODETTAGLIO'].str.upper()

In [9]:
# Retrieve only the year from the date
attivati['DATA'] = pd.to_datetime(attivati['DATA'], format="%d/%m/%Y")
attivati['mese-anno'] = attivati['DATA'].apply(lambda x: x.strftime('%Y-%m'))
attivati ['ANNO'] = attivati.DATA.dt.year
attivati = attivati.drop(labels='DATA', axis=1)

In [10]:
# Join between the two DataFrames
join = pd.merge(attivati, ateco, on =['SETTOREECONOMICODETTAGLIO'], how='inner')
join

,GENERE,ETA,SETTOREECONOMICODETTAGLIO,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,mese-anno,ANNO,Codice_ateco
0,F,60,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,UCRAINA,2020-05,2020,97
1,F,33,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO A TEMPO DETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012-07,2012,97
2,F,45,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,ITALIA,2019-04,2019,97
3,F,61,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,LECCO,UCRAINA,2014-09,2014,97
4,F,20,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014-05,2014,97
...,...,...,...,...,...,...,...,...,...,...,...
9329690,M,38,TRASPORTO MEDIANTE CONDOTTE DI LIQUIDI,LICENZA MEDIA,LAVORO A TEMPO INDETERMINATO,TEMPO PIENO,MILANO,ITALIA,2020-06,2020,49.50.2
9329691,F,40,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,COLLABORAZIONE COORDINATA E CONTINUATIVA,NON DEFINITO,BRESCIA,ITALIA,2021-11,2021,93.19.91
9329692,M,28,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,LICENZA MEDIA,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,MONZA E BRIANZA,ITALIA,2012-08,2012,93.19.91
9329693,F,25,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,COLLABORAZIONE COORDINATA E CONTINUATIVA,NON DEFINITO,BRESCIA,ITALIA,2018-11,2018,93.19.91


In [11]:
# Obtain only the first two digits of the 'codice ateco' to understand the macro category
join.Codice_ateco.replace(r'^(..)\..*' , r'\1',regex=True, inplace = True,)
join

,GENERE,ETA,SETTOREECONOMICODETTAGLIO,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,mese-anno,ANNO,Codice_ateco
0,F,60,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,UCRAINA,2020-05,2020,97
1,F,33,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO A TEMPO DETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012-07,2012,97
2,F,45,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,ITALIA,2019-04,2019,97
3,F,61,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,LECCO,UCRAINA,2014-09,2014,97
4,F,20,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014-05,2014,97
...,...,...,...,...,...,...,...,...,...,...,...
9329690,M,38,TRASPORTO MEDIANTE CONDOTTE DI LIQUIDI,LICENZA MEDIA,LAVORO A TEMPO INDETERMINATO,TEMPO PIENO,MILANO,ITALIA,2020-06,2020,49
9329691,F,40,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,COLLABORAZIONE COORDINATA E CONTINUATIVA,NON DEFINITO,BRESCIA,ITALIA,2021-11,2021,93
9329692,M,28,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,LICENZA MEDIA,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,MONZA E BRIANZA,ITALIA,2012-08,2012,93
9329693,F,25,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,COLLABORAZIONE COORDINATA E CONTINUATIVA,NON DEFINITO,BRESCIA,ITALIA,2018-11,2018,93


In [12]:
# Extract macro category
join2 = pd.merge(join, ateco, on='Codice_ateco', how='inner')
join2

,GENERE,ETA,SETTOREECONOMICODETTAGLIO_x,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,mese-anno,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y
0,F,60,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,UCRAINA,2020-05,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
1,F,33,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO A TEMPO DETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012-07,2012,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
2,F,45,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,ITALIA,2019-04,2019,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
3,F,61,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,LECCO,UCRAINA,2014-09,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
4,F,20,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014-05,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
...,...,...,...,...,...,...,...,...,...,...,...,...
9329690,M,36,INDUSTRIA DEL TABACCO,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,BERGAMO,ITALIA,2012-07,2012,12,INDUSTRIA DEL TABACCO
9329691,M,35,INDUSTRIA DEL TABACCO,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BRESCIA,ITALIA,2011-07,2011,12,INDUSTRIA DEL TABACCO
9329692,M,27,INDUSTRIA DEL TABACCO,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,MILANO,ITALIA,2009-04,2009,12,INDUSTRIA DEL TABACCO
9329693,M,37,INDUSTRIA DEL TABACCO,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,BERGAMO,ITALIA,2012-06,2012,12,INDUSTRIA DEL TABACCO


In [13]:
# Delete the sub category
join2 = join2.drop(labels='SETTOREECONOMICODETTAGLIO_x', axis=1)

In [14]:
# Transformations of the strings in upper case, to make a match, without problem
join2['CONTRATTO']=join2['CONTRATTO'].str.upper()
join2

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,mese-anno,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y
0,F,60,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,UCRAINA,2020-05,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
1,F,33,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO A TEMPO DETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012-07,2012,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
2,F,45,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,ITALIA,2019-04,2019,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
3,F,61,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,LECCO,UCRAINA,2014-09,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
4,F,20,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014-05,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
...,...,...,...,...,...,...,...,...,...,...,...
9329690,M,36,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,BERGAMO,ITALIA,2012-07,2012,12,INDUSTRIA DEL TABACCO
9329691,M,35,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BRESCIA,ITALIA,2011-07,2011,12,INDUSTRIA DEL TABACCO
9329692,M,27,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,MILANO,ITALIA,2009-04,2009,12,INDUSTRIA DEL TABACCO
9329693,M,37,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,BERGAMO,ITALIA,2012-06,2012,12,INDUSTRIA DEL TABACCO


In [15]:
# Set generic category for the contracts 
join2.loc[join2['CONTRATTO'].str.match(r'.*INDETERMINATO.*') == True,"CONTRATTO"]="INDETERMINATO"
join2.loc[join2['CONTRATTO'].str.match(r'.*INDETERMINATO.*') == False,"CONTRATTO"]="NON INDETERMINATO"

indeterminato=join2
indeterminato

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,mese-anno,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y
0,F,60,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,UCRAINA,2020-05,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
1,F,33,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012-07,2012,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
2,F,45,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,ITALIA,2019-04,2019,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
3,F,61,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,LECCO,UCRAINA,2014-09,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
4,F,20,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014-05,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
...,...,...,...,...,...,...,...,...,...,...,...
9329690,M,36,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,NON INDETERMINATO,NaN,BERGAMO,ITALIA,2012-07,2012,12,INDUSTRIA DEL TABACCO
9329691,M,35,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BRESCIA,ITALIA,2011-07,2011,12,INDUSTRIA DEL TABACCO
9329692,M,27,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,MILANO,ITALIA,2009-04,2009,12,INDUSTRIA DEL TABACCO
9329693,M,37,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,NON INDETERMINATO,NaN,BERGAMO,ITALIA,2012-06,2012,12,INDUSTRIA DEL TABACCO


In [16]:
indeterminato['ITALIANO'].unique()

array(['UCRAINA', 'HONDURAS', 'ITALIA', 'ECUADOR', 'ROMANIA', 'SENEGAL',
       'BOLIVIA', 'BRASILE', 'FILIPPINE', 'Cinese, Repubblica Popolare',
       "COSTA D'AVORIO", 'LIBANO', 'EL SALVADOR', 'MAROCCO', 'Moldova',
       'POLONIA', 'NIGERIA', 'SRI LANKA', 'PAKISTAN', 'INDIA', "PERU'",
       'GEORGIA', 'APOLIDE', 'BANGLADESH', 'EGITTO', 'ALBANIA', 'TUNISIA',
       'TURCHIA', 'MALI', 'GUINEA', 'COLOMBIA', 'BURKINA FASO',
       'RUSSA, Federazione', 'ALGERIA', 'BULGARIA', 'GIAMAICA', 'BENIN',
       'CUBA', 'GHANA', 'Dominicana, Repubblica', 'BIELORUSSIA',
       'CAMERUN', 'MACEDONIA, ex REP. JUGOSLAVIA', 'KENYA', 'SUDAN',
       'Mauritius', 'BOSNIA-ERZEGOVINA', 'VENEZUELA', 'SIRIA', 'TAIWAN',
       'SERBIA', 'UNGHERIA', 'GIAPPONE', 'CROAZIA', 'SPAGNA', 'DOMINICA',
       'ETIOPIA', 'TOGO', 'Congo, Rep.Democratica', 'PORTOGALLO',
       'MAURITANIA', "STATI UNITI D'AMERICA", 'LITUANIA', 'NEPAL',
       'GERMANIA', 'GUINEA EQUATORIALE', 'FRANCIA', 'CONGO', 'PARAGUAY',
       'EST

In [17]:
africa=pd.DataFrame(["SENEGAL ","NIGERIA","MAROCCO","EGITTO","GUINEA","GHANA","Mauritius","ALGERIA","ETIOPIA","TUNISIA","COSTA D’AVORIO","CAMERUN","GUINEA EQUATORIALE","BENIN","MAURITANIA","ARGENTINA","TOGO","UGANDA","CONGO","NEPAL","ERITREA","Congo, Rep.Democratica","SIERRA LEONE","SOMALIA","GAMBIA","NIGER","KENYA","LIBIA","SEYCHELLES","SUD AFRICA","CAPO VERDE","BURUNDI","TANZANIA","GABON","ZIMBABWE","SUD SUDAN","GUINEA BISSAU","LIBERIA","ZAMBIA","ANGOLA","MADAGASCAR","SUDAN","RUANDA","Centrafricana, Repubblica","CIAD","MOZAMBICO","Sao Tomè e Principe","SWAZILAND","GUYANA","NAMIBIA"])
africa['merge']='AFRICA'
sudame=pd.DataFrame(["BOLIVIA ","ECUADOR ","PERU","COLOMBIA","BRASILE","VENEZUELA","PARAGUAY","CILE","MALI","URUGUAY","TRINIDAD E TOBAGO","SAINT LUCIA"])
sudame['merge']='SUD AMERICA'
centame=pd.DataFrame(["EL SALVADOR","Dominicana, Repubblica","CUBA","DOMINICA","BAHAMAS","HONDURAS","GUATEMALA","PANAMA","NICARAGUA","HAITI","BELIZE","ANTIGUA E BARBUDA","COSTA RICA","Saint Vincent e Grenadin"])
centame['merge']='CENTRO AMERICA'
asia=pd.DataFrame(["FILIPPINE","BANGLADESH","GEORGIA","PAKISTAN","INDIA","BURKINA FASO","KAZAKHSTAN","SIRIA","THAILANDIA","KIRGHIZISTAN","INDONESIA","AFGHANISTAN","AZERBAIGIAN","UZBEKISTAN","CAMBOGIA","MYANMAR","TAGIKISTAN","VIETNAM","ARMENIA","Corea, Repubblica Popolare","TUVALU","MONGOLIA","LAOS","MALAYSIA","TURKMENISTAN","VANUATU","PALAU","SINGAPORE","Corea, Repubblica","BHUTAN","Timor Orientale","TONGA","KIRIBATI","MALDIVE","BRUNEI","PAPUA NUOVA GUINEA","Micronesia, Stati Federati","NAURU","FIGI"])
asia['merge']='ASIA'
medioor=pd.DataFrame(["LIBANO","IRAN","ISRAELE","TURCHIA","YEMEN","BAHREIN","GIORDANIA","QATAR","IRAQ","Territori Autonomia Palestinese","ARABIA SAUDITA","OMAN","KUWAIT","EMIRATI ARABI UNITI","TERRITORI PALESTINESI"])
medioor['merge']='MEDIO ORIENTE'
singoli=pd.DataFrame(["TAIWAN" ,"Cinese, Repubblica Popolare","ROMANIA","Moldova","UCRAINA","SRI LANKA","APOLIDE","ITALIA","ALBANIA","RUSSA, Federazione","POLONIA","BOSNIA-ERZEGOVINA","UNGHERIA","GIAPPONE","MACEDONIA","ex REP. JUGOSLAVIA","CROAZIA","BULGARIA","SPAGNA","BIELORUSSIA","SERBIA E MONTENEGRO","FRANCIA","MONTENEGRO","LETTONIA","REGNO UNITO","AUSTRALIA","KOSOVO","LITUANIA","GERMANIA","ESTONIA","IRLANDA","MESSICO","PORTOGALLO","Marshall, Isole","DANIMARCA","GIAMAICA","STATI UNITI D’AMERICA","PAESI BASSI","CECA, Repubblica","BELGIO","SVIZZERA","SLOVENIA","CANADA","NORVEGIA","SLOVACCHIA","GRECIA","CIPRO","FINLANDIA","SVEZIA","MONACO","ANDORRA","MALTA","AUSTRIA","LUSSEMBURGO","ISLANDA","NUOVA ZELANDA","SAN MARINO"])
singoli=pd.concat([singoli,singoli], axis=1)
singoli.columns = [0,'merge']
naz=pd.concat([africa,sudame,centame,asia,medioor,singoli], ignore_index=True)
naz.columns = ['ITALIANO','merge']
naz

,ITALIANO,merge
0,SENEGAL,AFRICA
1,NIGERIA,AFRICA
2,MAROCCO,AFRICA
3,EGITTO,AFRICA
4,GUINEA,AFRICA
...,...,...
182,AUSTRIA,AUSTRIA
183,LUSSEMBURGO,LUSSEMBURGO
184,ISLANDA,ISLANDA
185,NUOVA ZELANDA,NUOVA ZELANDA


In [18]:
join =pd.merge(indeterminato, naz,on=['ITALIANO'],how='inner')
join

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,mese-anno,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y,merge
0,F,60,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,UCRAINA,2020-05,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
1,F,61,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,LECCO,UCRAINA,2014-09,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
2,F,29,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,LECCO,UCRAINA,2017-06,2017,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
3,F,48,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,UCRAINA,2017-08,2017,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
4,F,54,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,UCRAINA,2020-01,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
...,...,...,...,...,...,...,...,...,...,...,...,...
9007130,M,32,LAUREA - Vecchio o nuovo ordinamento,INDETERMINATO,TEMPO PIENO,MILANO,SWAZILAND,2021-08,2021,47,COMMERCIO AL DETTAGLIO (ESCLUSO QUELLO DI AUTO...,AFRICA
9007131,M,28,LICENZA MEDIA,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,MILANO,SWAZILAND,2013-06,2013,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA
9007132,M,29,LICENZA MEDIA,NON INDETERMINATO,TEMPO PIENO,MILANO,SWAZILAND,2014-07,2014,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA
9007133,M,52,LICENZA MEDIA,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,MILANO,SWAZILAND,2018-12,2018,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA


In [19]:
join = join.drop(columns=["ITALIANO"])
join

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,mese-anno,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y,merge
0,F,60,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,2020-05,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
1,F,61,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,LECCO,2014-09,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
2,F,29,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,LECCO,2017-06,2017,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
3,F,48,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,2017-08,2017,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
4,F,54,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,2020-01,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
...,...,...,...,...,...,...,...,...,...,...,...
9007130,M,32,LAUREA - Vecchio o nuovo ordinamento,INDETERMINATO,TEMPO PIENO,MILANO,2021-08,2021,47,COMMERCIO AL DETTAGLIO (ESCLUSO QUELLO DI AUTO...,AFRICA
9007131,M,28,LICENZA MEDIA,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,MILANO,2013-06,2013,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA
9007132,M,29,LICENZA MEDIA,NON INDETERMINATO,TEMPO PIENO,MILANO,2014-07,2014,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA
9007133,M,52,LICENZA MEDIA,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,MILANO,2018-12,2018,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA


In [20]:
join.rename(columns={'merge':'ITALIANO'}, inplace=True)
indeterminato=join.copy()
indeterminato

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,mese-anno,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y,ITALIANO
0,F,60,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,2020-05,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
1,F,61,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,LECCO,2014-09,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
2,F,29,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,LECCO,2017-06,2017,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
3,F,48,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,2017-08,2017,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
4,F,54,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,2020-01,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA
...,...,...,...,...,...,...,...,...,...,...,...
9007130,M,32,LAUREA - Vecchio o nuovo ordinamento,INDETERMINATO,TEMPO PIENO,MILANO,2021-08,2021,47,COMMERCIO AL DETTAGLIO (ESCLUSO QUELLO DI AUTO...,AFRICA
9007131,M,28,LICENZA MEDIA,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,MILANO,2013-06,2013,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA
9007132,M,29,LICENZA MEDIA,NON INDETERMINATO,TEMPO PIENO,MILANO,2014-07,2014,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA
9007133,M,52,LICENZA MEDIA,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,MILANO,2018-12,2018,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA


#### Label Encoder

In [21]:
# Starting the procedure to encode the data
titolostudio = indeterminato.TITOLOSTUDIO.tolist()
modlavoro = indeterminato.MODALITALAVORO.tolist()
prov = indeterminato.PROVINCIAIMPRESA.tolist()
nazionalita = indeterminato.ITALIANO.tolist()
copia = indeterminato.copy()

In [22]:
# Instantiate labelencoder object for each column that we want to encode
le_titolostudio = LabelEncoder()
le_modlavoro = LabelEncoder()
le_prov = LabelEncoder()
le_nazionalita = LabelEncoder()

In [23]:
# Apply the label encoder to the dataset
le_titolostudio.fit(copia['TITOLOSTUDIO'])
le_modlavoro.fit(copia['MODALITALAVORO'])
le_prov.fit(copia['PROVINCIAIMPRESA'])
le_nazionalita.fit(copia['ITALIANO'])

LabelEncoder()

In [24]:
# Transform into integers
transformation_ts = le_titolostudio.transform(indeterminato['TITOLOSTUDIO']) 
transformation_ts

array([8, 8, 8, ..., 6, 6, 6])

In [25]:
# Try to invert a transformation using the object
list(le_titolostudio.inverse_transform([8]))

['NESSUN TITOLO DI STUDIO']

In [26]:
# Transform into integers
transformation_ml = le_modlavoro.transform(indeterminato['MODALITALAVORO']) 
transformation_prov = le_prov.transform(indeterminato['PROVINCIAIMPRESA']) 
transformation_nazion = le_nazionalita.transform(indeterminato['ITALIANO']) 

In [28]:
le_nazionalita.transform(['ITALIA'])

array([27])

In [27]:
# Apply label encoder on categorical feature columns
copia['titolostudio_transformed'] = list(transformation_ts)
copia['modalitalavoro_transformed'] = list(transformation_ml)
copia['provincia_transformed'] = list(transformation_prov)
copia['nazionalita_transformed'] = list(transformation_nazion)
copia

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,mese-anno,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y,ITALIANO,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,nazionalita_transformed
0,F,60,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,2020-05,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,4,0,57
1,F,61,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,LECCO,2014-09,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,4,4,57
2,F,29,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,LECCO,2017-06,2017,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,2,4,57
3,F,48,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,2017-08,2017,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,2,1,57
4,F,54,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,2020-01,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,2,1,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9007130,M,32,LAUREA - Vecchio o nuovo ordinamento,INDETERMINATO,TEMPO PIENO,MILANO,2021-08,2021,47,COMMERCIO AL DETTAGLIO (ESCLUSO QUELLO DI AUTO...,AFRICA,4,4,7,0
9007131,M,28,LICENZA MEDIA,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,MILANO,2013-06,2013,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA,6,2,7,0
9007132,M,29,LICENZA MEDIA,NON INDETERMINATO,TEMPO PIENO,MILANO,2014-07,2014,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA,6,4,7,0
9007133,M,52,LICENZA MEDIA,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,MILANO,2018-12,2018,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA,6,2,7,0


#### Encoding Manually Ordinal Categorical Features

In [28]:
# Create mapper
scale_mapper = {'NON INDETERMINATO':0, 'INDETERMINATO':1}

# Map feature values to scale
copia['contratto_transformed'] = copia['CONTRATTO'].replace(scale_mapper)
copia

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,mese-anno,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y,ITALIANO,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,nazionalita_transformed,contratto_transformed
0,F,60,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,2020-05,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,4,0,57,0
1,F,61,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,LECCO,2014-09,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,4,4,57,0
2,F,29,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,LECCO,2017-06,2017,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,2,4,57,0
3,F,48,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,2017-08,2017,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,2,1,57,0
4,F,54,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,2020-01,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,2,1,57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9007130,M,32,LAUREA - Vecchio o nuovo ordinamento,INDETERMINATO,TEMPO PIENO,MILANO,2021-08,2021,47,COMMERCIO AL DETTAGLIO (ESCLUSO QUELLO DI AUTO...,AFRICA,4,4,7,0,1
9007131,M,28,LICENZA MEDIA,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,MILANO,2013-06,2013,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA,6,2,7,0,0
9007132,M,29,LICENZA MEDIA,NON INDETERMINATO,TEMPO PIENO,MILANO,2014-07,2014,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA,6,4,7,0,0
9007133,M,52,LICENZA MEDIA,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,MILANO,2018-12,2018,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA,6,2,7,0,0


In [29]:
# Create mapper
scale_mapper = {'M':0, 'F':1}

# Map feature values to scale
copia['genere_transformed'] = copia['GENERE'].replace(scale_mapper)
copia

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,mese-anno,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y,ITALIANO,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,nazionalita_transformed,contratto_transformed,genere_transformed
0,F,60,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,2020-05,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,4,0,57,0,1
1,F,61,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,LECCO,2014-09,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,4,4,57,0,1
2,F,29,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,LECCO,2017-06,2017,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,2,4,57,0,1
3,F,48,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,2017-08,2017,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,2,1,57,0,1
4,F,54,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,2020-01,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,2,1,57,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9007130,M,32,LAUREA - Vecchio o nuovo ordinamento,INDETERMINATO,TEMPO PIENO,MILANO,2021-08,2021,47,COMMERCIO AL DETTAGLIO (ESCLUSO QUELLO DI AUTO...,AFRICA,4,4,7,0,1,0
9007131,M,28,LICENZA MEDIA,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,MILANO,2013-06,2013,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA,6,2,7,0,0,0
9007132,M,29,LICENZA MEDIA,NON INDETERMINATO,TEMPO PIENO,MILANO,2014-07,2014,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA,6,4,7,0,0,0
9007133,M,52,LICENZA MEDIA,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,MILANO,2018-12,2018,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA,6,2,7,0,0,0


In [30]:
# Trasform the column in int type
copia['Codice_ateco']=copia.Codice_ateco.astype(int)

# We remove the outliers. 
copia = copia[copia['ETA']>=15]
copia = copia[copia['ETA']<100]
copia = copia[copia['ANNO']<2022]
copia = copia[copia['ANNO']>2006]
copia

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,mese-anno,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y,ITALIANO,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,nazionalita_transformed,contratto_transformed,genere_transformed
0,F,60,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,2020-05,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,4,0,57,0,1
1,F,61,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,LECCO,2014-09,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,4,4,57,0,1
2,F,29,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,LECCO,2017-06,2017,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,2,4,57,0,1
3,F,48,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,2017-08,2017,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,2,1,57,0,1
4,F,54,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,2020-01,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,UCRAINA,8,2,1,57,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9007130,M,32,LAUREA - Vecchio o nuovo ordinamento,INDETERMINATO,TEMPO PIENO,MILANO,2021-08,2021,47,COMMERCIO AL DETTAGLIO (ESCLUSO QUELLO DI AUTO...,AFRICA,4,4,7,0,1,0
9007131,M,28,LICENZA MEDIA,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,MILANO,2013-06,2013,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA,6,2,7,0,0,0
9007132,M,29,LICENZA MEDIA,NON INDETERMINATO,TEMPO PIENO,MILANO,2014-07,2014,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA,6,4,7,0,0,0
9007133,M,52,LICENZA MEDIA,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,MILANO,2018-12,2018,52,MAGAZZINAGGIO E ATTIVITÀ DI SUPPORTO AI TRASPORTI,AFRICA,6,2,7,0,0,0


In [31]:
# Drop the columns that have been transformed
transformed_indet = copia.drop(labels =['GENERE', 'TITOLOSTUDIO', 'MODALITALAVORO', 'PROVINCIAIMPRESA', 'ITALIANO', 'SETTOREECONOMICODETTAGLIO_y', 'CONTRATTO'], axis=1)
transformed_indet

,ETA,mese-anno,ANNO,Codice_ateco,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,nazionalita_transformed,contratto_transformed,genere_transformed
0,60,2020-05,2020,97,8,4,0,57,0,1
1,61,2014-09,2014,97,8,4,4,57,0,1
2,29,2017-06,2017,97,8,2,4,57,0,1
3,48,2017-08,2017,97,8,2,1,57,0,1
4,54,2020-01,2020,97,8,2,1,57,0,1
...,...,...,...,...,...,...,...,...,...,...
9007130,32,2021-08,2021,47,4,4,7,0,1,0
9007131,28,2013-06,2013,52,6,2,7,0,0,0
9007132,29,2014-07,2014,52,6,4,7,0,0,0
9007133,52,2018-12,2018,52,6,2,7,0,0,0


In [32]:
# Change the type of all column
transformed_indet['Codice_ateco'] = transformed_indet['Codice_ateco'].astype(str).astype(int)
transformed_indet.dtypes

ETA                            int64
mese-anno                     object
ANNO                           int64
Codice_ateco                   int32
titolostudio_transformed       int32
modalitalavoro_transformed     int32
provincia_transformed          int32
nazionalita_transformed        int32
contratto_transformed          int64
genere_transformed             int64
dtype: object

In [33]:
# Encoding of the output
transformed_indet=trasformed_input(transformed_indet,'Output')
transformed_indet

,ETA,mese-anno,ANNO,Codice_ateco,nazionalita_transformed,contratto_transformed,genere_transformed,Output
0,60,2020-05,2020,97,57,0,1,97 - 8 - 4 - 0
1,61,2014-09,2014,97,57,0,1,97 - 8 - 4 - 4
2,29,2017-06,2017,97,57,0,1,97 - 8 - 2 - 4
3,48,2017-08,2017,97,57,0,1,97 - 8 - 2 - 1
4,54,2020-01,2020,97,57,0,1,97 - 8 - 2 - 1
...,...,...,...,...,...,...,...,...
9007130,32,2021-08,2021,47,0,1,0,47 - 4 - 4 - 7
9007131,28,2013-06,2013,52,0,0,0,52 - 6 - 2 - 7
9007132,29,2014-07,2014,52,0,0,0,52 - 6 - 4 - 7
9007133,52,2018-12,2018,52,0,0,0,52 - 6 - 2 - 7


In [34]:
# With LabelEncoder we encode the output column
copia = transformed_indet.copy()
le_output = LabelEncoder()
le_output.fit(copia['Output'])
transformation_output = le_output.transform(transformed_indet['Output']) 
copia['Output_transformed'] = list(transformation_output)
copia=copia.drop(labels =['Output'], axis=1)
copia

,ETA,mese-anno,ANNO,Codice_ateco,nazionalita_transformed,contratto_transformed,genere_transformed,Output_transformed
0,60,2020-05,2020,97,57,0,1,32470
1,61,2014-09,2014,97,57,0,1,32476
2,29,2017-06,2017,97,57,0,1,32455
3,48,2017-08,2017,97,57,0,1,32450
4,54,2020-01,2020,97,57,0,1,32450
...,...,...,...,...,...,...,...,...
9007130,32,2021-08,2021,47,0,1,0,14264
9007131,28,2013-06,2013,52,0,0,0,15697
9007132,29,2014-07,2014,52,0,0,0,15721
9007133,52,2018-12,2018,52,0,0,0,15697


In [35]:
copia.to_csv('transformed.csv')